# Getting started with the practicals

***These notebooks are best viewed in Jupyter. GitHub might not display all content of the notebook properly.***

## Goal of the practical exercises

The exercises have two goals:

1. Give you the opportunity to obtain 'hands-on' experience in implementing, training and evaluation machine learning models in Python. This experience will also help you better understand the theory covered during the lectures. 

2. Occasionally demonstrate some 'exam-style' questions that you can use as a reference when studying for the exam. Note however that the example questions are (as the name suggests) only examples and do not constitute a complete and sufficient list of 'things that you have to learn for the exam'. You can recognize example questions as (parts of) exercises by <font color="#770a0a">this font color</font>.

For each set of exercises (one Python notebook such as this one $==$ one set of exercises) you have to submit deliverables that will then be graded and constitute 25% of the final grade. Thus, the work that you do during the practicals has double contribution towards the final grade: as 25% direct contribution and as a preparation for the exam that will define the other 65% of the grade.

## Deliverables

For each set of exercises, you have to submit:
1. Python functions and/or classes (`.py` files) that implement basic functionalities (e.g. a $k$-NN classifier) and 
2. A *single* Python notebook that contains the experiments, visualization and answer to the questions and math problems. *Do not submit your answers as Word or PDF documents (they will not be graded)*. The submitted code and notebook should run without errors and be able to fully reproduce the reported results.

We recommend that you clone the provided notebooks (such as this one) and write your code in them. The following rubric will be used when grading the practical work:

Component  | Insufficient | Satisfactory | Excellent
--- | --- | --- | ---
**Code** | Missing or incomplete code structure, runs with errors, lacks documentation | Self-contained, does not result in errors, contains some documentation, can be easily used to reproduce the reported results | User-friendly, well-structured (good separation of general functionality and experiments, i.e. between `.py` files and the Pyhthon notebook), detailed documentation, optimized for speed, <s>use of a version control system (such as GitHub)</s>
**Answers to questions** | Incorrect, does not convey understanding of the material, appears to be copied from another source | Correct, conveys good understanding of the material, description in own words | Correct, conveys excellent level of understanding, makes connections between topics

## A word on notation

When we refer to Python variables, we will use a monospace font. For example, `X` is a Python variable that contains the data matrix. When we refer to mathematical variables, we will use the de-facto standard notation: $a$ or $\lambda$ is a scalar variable, $\boldsymbol{\mathrm{w}}$ is a vector and $\boldsymbol{\mathrm{X}}$ is a matrix (e.g. a data matrix from the example above). You should use the same notation when writing your answers and solutions.

# Two simple machine learning models

## Preliminaries

Throughout the practical curriculum of this course, we will use the Python programming language and its ecosystem of libraries for scientific computing (such as `numpy`, `scipy`, `matplotlib`, `scikit-learn` etc). The practicals for the deep learning part of the course will use the `keras` deep learning framework. If you are not sufficiently familiar with this programming language and/or the listed libraries and packages, you are strongly advised to go over the corresponding tutorials from the ['Essential skills'](https://github.com/tueimage/essential-skills) module (the `scikit-learn` library is not covered by the tutorial, however, an extensive documentation is available [here](https://scikit-learn.org/stable/documentation.html).

In this first set of exercises, we will use two toy datasets that ship together with `scikit-learn`. 

The first dataset is named `diabetes` and contains 442 patients described with 10 features: age, sex, body mass index, average blood pressure, and six blood serum measurements. The target variable is a continuous quantitative measure of the disease (diabetes) progression one year after the baseline measurements were recorded. More information is available [here](https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/datasets/descr/diabetes.rst) and [here](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html).

The second dataset is named `breast_cancer` and is a copy of the UCI ML Breast Cancer Wisconsin (Diagnostic) datasets (more infortmation is available [here](https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/datasets/descr/breast_cancer.rst) and [here](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)). The datasets contains of 569 instances represented with 30 features that are computed from a images of a fine needle aspirate of a breast mass. The features describe characteristics of the cell nuclei present in the image. Each instance is associated with a binary target variable ('malignant' or 'benign'). 

You can load the two datasets in the following way:

In [1]:
import numpy as np
from sklearn.datasets import load_diabetes, load_breast_cancer
import matplotlib.pyplot as plt
from scipy.stats import norm
plt.rcParams.update({'figure.max_open_warning': 0})


diabetes = load_diabetes()

breast_cancer = load_breast_cancer()

ModuleNotFoundError: No module named 'sklearn'

In the majority of the exercises in this course, we will use higher-level libraries and packages such as `scikit-learn` and `keras` to implement, train and evaluate machine learning models. However, the goal of this first set of exercises is to illustrate basic mathematical tools and machine learning concepts. Because of this, we will impose a restriction of only using basic `numpy` functionality. Furthermore, you should as much as possible restrict the use of for-loops (e.g. use a vector-to-matrix product instead of a for loop when appropriate).

If `X` is a 2D data matrix, we will use the convention that the rows of the matrix contain the samples (or instances) and the columns contain the features (inputs to the model). That means that a data matrix with a shape `(122, 13)` represents a dataset with 122 samples, each represented with 13 features. Similarly, if `Y` is a 2D matrix containing the targets, the rows correspond to the samples and the columns to the different targets (outputs of the model). Thus, if the shape of `Y` is `(122, 3)` that means that there are 122 samples and each sample is has 3 targets (note that in the majority of the examples we will only have a single target and thus the number of columns of `Y` will be 1).

You can obtain the data and target matrices from the two datasets in the following way:

In [ ]:
X = diabetes.data
print(diabetes.target.shape)
Y = diabetes.target[:, np.newaxis]

print(X.shape)
print(Y.shape)

If you want to only use a subset of the available features, you can obtain a reduced data matrix in the following way:

In [ ]:
# use only the fourth feature
X = diabetes.data[:, np.newaxis, 3]
print(X.shape)

# use the third, and tenth features
X = diabetes.data[:, (3,9)]
print(X.shape)

***Question***: Why we need to use the `np.newaxis` expression in the examples above? 

Note that in all your experiments in the exercises, you should use and independent training and testing sets. You can split the dataset into a training and testing subsets in the following way:

**Answer** : For the first example, we use np.newaxis to make sure that Y is, just like X, also 2 dimensional since Y first only has shape (442, ) . To make vector operations possible we need to make sure that Y also has two dimensions like X by specifying a new axis with length 1 since Y is a vector with the target labels. For the second example, we are only selecting the fourth feature of X which would also result in a column but with a shape of (442,). To again make sure that it is 2 dimensional and compatible for matrix operations with Y, we have to specify a new axis with length 1 so that it is still a column but now two dimensional with 442 rows and 1 column.

In [ ]:
# use the fourth feature
# use the first 300 training samples for training, and the rest for testing
X_train = diabetes.data[:300, np.newaxis, 3]
y_train = diabetes.target[:300, np.newaxis]
X_test = diabetes.data[300:, np.newaxis, 3]
y_test = diabetes.target[300:, np.newaxis]
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

## Exercises

### Linear regression

Implement training and evaluation of a linear regression model on the diabetes dataset using only matrix multiplication, inversion and transpose operations. Report the mean squared error of the model.

To get you started we have implemented the first part of this exercise (fitting of the model) as an example.

In [ ]:
# add subfolder that contains all the function implementations
# to the system path so we can import them
import sys
sys.path.append('code/')

# the actual implementation is in linear_regression.py,
# here we will just use it to fit a model
from linear_regression_myrthe import *

# load the dataset
# same as before, but now we use all features
X_train = diabetes.data[:300, :]
y_train = diabetes.target[:300, np.newaxis]
X_test = diabetes.data[300:, :]
y_test = diabetes.target[300:, np.newaxis]

beta = lsq(X_train, y_train)

# print the parameters
#print(beta)

# Predict on the test set

ones = np.ones((len(X_test), 1))
X_test = np.concatenate((ones, X_test), axis=1)
y_pred = np.dot(X_test, beta)

lin_reg_mse = mse(y_pred, y_test)
print(mse(y_pred, y_test))



### Weighted linear regression

Assume that in the dataset that you use to train a linear regression model, there are identical versions of some samples. This problem can be reformulated to a weighted linear regression problem where the matrices $\boldsymbol{\mathrm{X}}$ and $\boldsymbol{\mathrm{Y}}$ (or the vector $\boldsymbol{\mathrm{y}}$ if there is only a single target/output variable) contain only the unique data samples, and a vector $\boldsymbol{\mathrm{d}}$ is introduced that gives more weight to samples that appear multiple times in the original dataset (for example, the sample that appears 3 times has a corresponding weight of 3). 

<p><font color='#770a0a'>Derive the expression for the least-squares solution of a weighted linear regression model (note that in addition to the matrices $\boldsymbol{\mathrm{X}}$ and $\boldsymbol{\mathrm{Y}}$, the solution should include a vector of weights $\boldsymbol{\mathrm{d}}$).</font></p>

### $k$-NN classification

Implement a $k$-Nearest neighbors classifier from scratch in Python using only basic matrix operations with `numpy` and `scipy`. Train and evaluate the classifier on the breast cancer dataset, using all features. Show the performance of the classifier for different values of $k$ (plot the results in a graph). Note that for optimal results, you should normalize the features (e.g. to the $[0, 1]$ range or to have a zero mean and unit standard deviation).

In [ ]:
# Normalize the features by subtracting the mean of complete dataset and divide by standard deviation
X = breast_cancer.data
X = (X - np.mean(X)) / (np.std(X))

# Split in training and test set
X_train = X[:300]
y_train = breast_cancer.target[:300]
X_test = X[300:]
y_test = breast_cancer.target[300:]

# Calculate accuracy (performance metric, number of correct positive predictions) for different values of k

acc_list = []

for k in range(1,40):
    pred = kNN(X_train, y_train, X_test, y_test, k, regression=False)
    N = pred.shape[0]
    accuracy = (y_test == pred).sum() / N
    acc_list.append(accuracy)

plt.plot(np.arange(1,40), acc_list)
plt.title('kNN - Accuracy for different values of k')
plt.xlabel('k')
plt.ylabel('accuracy')


### $k$-NN regression

Modify the $k$-NN implementation to do regression instead of classification. Compare the performance of the linear regression model and the $k$-NN regression model on the diabetes dataset for different values of $k$..

In [ ]:
# load the diabetes dataset + split in training and test
X_train = diabetes.data[:300]
y_train = diabetes.target[:300]
X_test = diabetes.data[300:]
y_test = diabetes.target[300:]

# Calculate MSE (performance metric) for different values of k

mse_list = []

for k in range(1,30):
    y_pred = kNN(X_train, y_train, X_test, y_test, k, regression=True)
    mse_knn = mse(y_pred, y_test)
    mse_list.append(mse_knn)

# We plot the MSE's for different values of k 
# Together with the MSE the linear regression yielded for the diabetes dataset

plt.plot(np.arange(1,30), mse_list)
legend= ['kNN', 'Linear Regression']
plt.axhline(y=lin_reg_mse, color='r', linestyle='-')
plt.title('MSE for different values of k - kNN')
plt.legend(legend)
plt.xlabel('k')
plt.ylabel('MSE')



### Class-conditional probability

Compute and visualize the class-conditional probability (conditional probability where the class label is the conditional variable, i.e. $P(X = x \mid Y = y_i)$ for all features in the breast cancer dataset. Assume a Gaussian distribution.

<p><font color='#770a0a'>Based on visual analysis of the plots, which individual feature can best discriminate between the two classes? Motivate your answer.</font></p>



In [ ]:
X = breast_cancer.data
y = breast_cancer.target


# We check the class conditional probability per feature

for i in range(X.shape[1]):
    
    # For each feature (i) we check what the values of that feature is for all samples that are classified as y = 1
    X_class1 = X[:, i][y==1]
    X_class0 = X[:, i][y==0]
    
    # We calculate the mean and std for the Gaussian distribution we can assume
    mu_1 = np.mean(X_class1)
    sigma_1 = np.std(X_class1)
    
    mu_0 = np.mean(X_class0)
    sigma_0 = np.std(X_class0)
    
    # We plot this distribution for each feature
    fig, ax = plt.subplots(1, 1)
    rv_1 = norm(mu_1, sigma_1)
    rv_0 = norm(mu_0, sigma_0)
    
    # Define x values based on mu and sigma
    x_1 = np.linspace(norm.ppf(0.01, loc=mu_1, scale=sigma_1),norm.ppf(0.99, loc=mu_1, scale=sigma_1), 100) 
    x_0 = np.linspace(norm.ppf(0.01, loc=mu_0, scale=sigma_0),norm.ppf(0.99, loc=mu_0, scale=sigma_0), 100) 
    ax.plot(x_0, rv_0.pdf(x_0), 'r-', lw=2, label='frozen pdf0')
    ax.plot(x_1, rv_1.pdf(x_1), 'b-', lw=2, label='frozen pdf1')
    legend = ['Conditional y = 0', 'Conditional y = 1']
    plt.legend(legend)
    plt.xlabel('feature_' + str(i) + ' values')
    plt.ylabel('Probability density')
    plt.title('Conditional probability density plots of feature_' + str(i), fontsize=10)

    
    
